### Task 1: Static Network Analysis
* Degree (in-degree/out-degree)
* Diameter
* Dyads
* Reciprocity and Transitivity

A temporal network is goint to be analysed which consists of 678907 vertices and 4729035 edges, where each edge has time information associated with it. Some of the edges have the same source and target vertex, but are association with different timestamps. This statick network analysi, however, ignores time-information and thus we use a graph build solely on the pairs of source and target vertices.

### <font color="darkgreen">Imports, configuation and preprocessing</font>

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import math
import igraph

#### Directed Graph with parallel edges (due to different timestamps of the edges between pairs of vertices)

In [2]:
wiki = open("data/tgraph_real_wikiedithyperlinks_noTime.txt", 'rb')
G_par = nx.read_edgelist(wiki, create_using=nx.MultiDiGraph())
wiki.close()

#### Directed Graph used for static network analysis (networkx)

In [3]:
wiki = open("data/tgraph_real_wikiedithyperlinks_noTime.txt", 'rb')
G = nx.read_edgelist(wiki, create_using=nx.DiGraph())
wiki.close()

#### Directed Graph used for static network analysis (igraph)

In [4]:
wiki = open("data/tgraph_real_wikiedithyperlinks_noTime.txt", 'rb')
G_i = igraph.read(wiki) # used for certain functions absenst in networkx / are more efficient in igraph
wiki.close()

In [5]:
def create_dataframe(G_in):
    df_edge_in = pd.DataFrame(list(G_in.in_degree()), columns=['node', 'in edges'])
    df_edge_out = pd.DataFrame(list(G_in.out_degree()), columns=['node', 'out edges'])
    df_total = pd.merge(df_edge_in, df_edge_out, on='node')
    df_total.index = df_total.index + 1
    return df_total

In [6]:
wiki_df_par = create_dataframe(G_par)

In [7]:
wiki_df = create_dataframe(G)
wiki_df.head()

,node,in edges,out edges
1,1,28,88
2,6,10,0
3,8,62,6
4,9,1375,477
5,3,1068,451


### <font color="darkgreen">1. Static Network Analysis:</font> Degree distribution

The first network property which is going to be analyzed of the static network is the degree distribution. While it is an easily comprehensible network measure, it can still lead to interesting findings as we have seen in the previous assignment.

In [8]:
wiki_df['in edges'].mean() == wiki_df['out edges'].mean()

True

Which is expected of course since every outgoing edge is an incoming edge of the graph. This check is just done to check whether the parsing of the Graph object to a dataframe is done without any errors.

In [9]:
wiki_df['in edges'].mean()

5.3091660566174745

In [10]:
wiki_df['in edges'].max(), wiki_df['in edges'].min()

(10521, 0)

In [11]:
wiki_df['out edges'].max(), wiki_df['out edges'].min()

(4302, 0)

The mean of the incoming edges is just under above the 5 links. The node with the highest number of incoming edges has a little more than $10^4$ incoming edges, while the node with the highest number of outgoing edges has around  $4.3*10^3$ outgoing edges. These values do not implicate anything by themselves, but these values will be used in elaborations further on.

Before we continue with the static network analysis note that for the static network analysis it does not make sense to consider parallel edges. The parallel edges are edges between a pair of vertices at different time-stamps. But this gives a deceptive view in the static network analysis part. If a page has $y$ incoming edges of the same source vertex $x$, at different time-stamps it does not make sense to consider all of these parallel edges for many measures such as the degree distribution, page-rank etc. The choice is, therefore, made to continue with the network as a directed graph ($DiGraph$ in networkx) instead of a multi-directed graph ($MultiDiGraph$) for the static network analysis. These edges, however, have to be considered in part two for the Temporal Network Analysis where the time information of the edges is of the essence and techniques such as snapshot-based analysis can or even should be exploited.

Just to give you an overview of the deceiving effects it can have on the measures calculated over these two types of networks:

In [12]:
wiki_df_par['in edges'].mean(), wiki_df_par['in edges'].max(), wiki_df_par['out edges'].max()

(6.965659508592488, 15871, 15017)

It adds up the parallel edges between pair of vertices which result in a higher number of incoming and/or outgoing edges for many vertices. 

Let's continue with the analysis of the degree-distribution now that we have this pecularity out of the way. We are going to divide nodes of the network into deciles to get a better grasp of the characteristics of the top and bottom nodes when it boils down to number of incoming or outgoing edges.

In [13]:
wiki_df['decile_incoming_edges'] = pd.cut((wiki_df['in edges']), 10, labels=False)
wiki_df.loc[wiki_df.decile_incoming_edges.between(1, 9)].shape, wiki_df.loc[wiki_df.decile_incoming_edges.between(0, 1)].shape

((220, 4), (678830, 4))

In [14]:
wiki_df['decile_outgoing_edges'] = pd.cut((wiki_df['out edges']), 10, labels=False)
wiki_df.loc[wiki_df.decile_outgoing_edges.between(1, 9)].shape, wiki_df.loc[wiki_df.decile_outgoing_edges.between(0, 1)].shape

((305, 5), (678835, 5))

We see that there is a very skewed distribution among the nodes when we make splits based upon their number number of incoming or outgoing edges. There is thus a small set of nodes (i.e. pages) with relatively high number of incoming edges and a small set of nodes with a relatively high number of outgoing edges.

Let's divide the nodes in four groups such that in each quartile we have the same number of vertices (not the same as quartiles) intstead of the current configuration since this will faciliate a better comparison of the the groups considering the very skewed distribution. 

In [15]:
sorted_incoming_edges = wiki_df.sort_values(['in edges']).reset_index(drop = True)
sorted_outgoing_edges = wiki_df.sort_values(['out edges']).reset_index(drop = True)
sorted_incoming_edges.tail()

,node,in edges,out edges,decile_incoming_edges,decile_outgoing_edges
678902,300,7975,1686,7,3
678903,3546,8561,0,8,0
678904,146,9264,0,8,0
678905,149,9655,545,9,1
678906,394,10521,1777,9,4


In [16]:
incoming_edges_q1 = sorted_incoming_edges.iloc[: math.floor(sorted_incoming_edges.shape[0] / 4)]
incoming_edges_q2 = sorted_incoming_edges.iloc[math.floor(sorted_incoming_edges.shape[0] / 4) : 2 * (math.floor(sorted_incoming_edges.shape[0] / 4))]
incoming_edges_q3 = sorted_incoming_edges.iloc[2 * (math.floor(sorted_incoming_edges.shape[0] / 4)) : 3 * (math.floor(sorted_incoming_edges.shape[0] / 4))]
incoming_edges_q4 = sorted_incoming_edges.iloc[3 * (math.floor(sorted_incoming_edges.shape[0] / 4)) :]

incoming_edges_bottom_1 = sorted_incoming_edges.iloc[: math.floor(sorted_incoming_edges.shape[0] / 100)]
incoming_edges_top_1 = sorted_incoming_edges.iloc[99 * math.floor(sorted_incoming_edges.shape[0] / 100):]

In [17]:
outgoing_edges_q1 = sorted_outgoing_edges.iloc[: math.floor(sorted_outgoing_edges.shape[0] / 4)]
outgoing_edges_q2 = sorted_outgoing_edges.iloc[math.floor(sorted_outgoing_edges.shape[0] / 4) : 2 * (math.floor(sorted_outgoing_edges.shape[0] / 4))]
outgoing_edges_q3 = sorted_outgoing_edges.iloc[2 * (math.floor(sorted_outgoing_edges.shape[0] / 4)) : 3 * (math.floor(sorted_outgoing_edges.shape[0] / 4))]
outgoing_edges_q4 = sorted_outgoing_edges.iloc[3 * (math.floor(sorted_outgoing_edges.shape[0] / 4)) :]

outgoing_edges_bottom_1 = sorted_outgoing_edges.iloc[: math.floor(sorted_outgoing_edges.shape[0] / 100)]
outgoing_edges_top_1 = sorted_outgoing_edges.iloc[99 * math.floor(sorted_outgoing_edges.shape[0] / 100):]

And just to double-check

In [18]:
(incoming_edges_q4.shape[0] + incoming_edges_q3.shape[0] + 
 incoming_edges_q2.shape[0] + incoming_edges_q1.shape[0]) == sorted_incoming_edges.shape[0]

True

Let's have a look at what there are some notable things in the devised groups

In [19]:
incoming_edges_q1['in edges'].mean(), incoming_edges_q2['in edges'].mean(), incoming_edges_q3['in edges'].mean(), incoming_edges_q4['in edges'].mean()

(0.0, 0.806753237571144, 1.8176236993742856, 18.61205215372741)

In [20]:
incoming_edges_q1['out edges'].mean(), incoming_edges_q2['out edges'].mean(), incoming_edges_q3['out edges'].mean(), incoming_edges_q4['out edges'].mean()

(3.8300142582751024, 2.36138835534921, 3.578950779491651, 11.466202004371675)

In [21]:
outgoing_edges_q1['in edges'].mean(), outgoing_edges_q2['in edges'].mean(), outgoing_edges_q3['in edges'].mean(), outgoing_edges_q4['in edges'].mean()

(3.9767507629944734, 3.667994296689959, 2.489977964483933, 11.101838813638212)

In [22]:
outgoing_edges_q1['out edges'].mean(), outgoing_edges_q2['out edges'].mean(), outgoing_edges_q3['out edges'].mean(), outgoing_edges_q4['out edges'].mean()

(0.0, 0.8123446024769334, 2.209997289749361, 18.214094232570744)

In [23]:
incoming_edges_bottom_1['in edges'].mean(), incoming_edges_top_1['in edges'].mean()

(0.0, 235.2441141848146)

In [24]:
incoming_edges_bottom_1['out edges'].mean(), incoming_edges_top_1['out edges'].mean()

(4.4613345117101195, 63.97998822836963)

In [25]:
outgoing_edges_bottom_1['in edges'].mean(), outgoing_edges_top_1['in edges'].mean()

(2.8858447488584473, 112.43952324896998)

In [26]:
outgoing_edges_bottom_1['out edges'].mean(), outgoing_edges_top_1['out edges'].mean()

(0.0, 160.2302825191289)

#### <i>Mean incoming and outgoing edges total network: $5.3$</i> 
#### <i>Sorted on number of incoming edges</i>
<table>
    <tr>
        <th>Group</th>
        <th>Mean #Incoming Edges</th>
        <th>Mean #Outgoing Edges</th>
    </tr>
    <tr>
        <td><i>Bottom 1%</i></td>
        <td>$0.00$</td>
        <td>$4.46$</td>
    </tr>
    <tr>
        <td>Q1 <i>(Bottom 25%)</i></td>
        <td>$0.00$</td>
        <td>$3.83$</td>
    </tr>
    <tr>
        <td>Q2</td>
        <td>$0.81$</td>
        <td>$2.36$</td>
    </tr>
    <tr>
        <td>Q3</td>
        <td>$1.82$</td>
        <td>$3.58$</td>
    </tr>
        <tr>
        <td>Q4 <i>(Top 25%)</i></td>
        <td>$18.6$</td>
        <td>$11.5$</td>
    </tr>
        <tr>
        <td><i>Top 1%</i></td>
        <td>$235$</td>
        <td>$64.0$</td>
    </tr>
</table>


#### <i>Sorted on number of outgoing edges</i>
<table>
    <tr>
        <th>Group</th>
        <th>Mean #Incoming Edges</th>
        <th>Mean #Outgoing Edges</th>
    </tr>
        <tr>
        <td><i>Bottom 1%</i></td>
        <td>$2.89$</td>
        <td>$0.00$</td>
    </tr>
    <tr>
        <td>Q1 <i>(Bottom 25%)</i></td>
        <td>$3.97$</td>
        <td>$0.00$</td>
    </tr>
    <tr>
        <td>Q2</td>
        <td>$3.67$</td>
        <td>$0.81$</td>
    </tr>
    <tr>
        <td>Q3</td>
        <td>$2.49$</td>
        <td>$2.21$</td>
    </tr>
        <tr>
        <td>Q4 <i>(Top 25%)</i></td>
        <td>$11.1$</td>
        <td>$18.2$</td>
    </tr>
        <tr>
        <td><i>Top 1%</i></td>
        <td>$112$</td>
        <td>$160$</td>
    </tr>
</table>


There are two noteworthy things in this graph. The nodes with a relatively high number of outgoing edges (i.e. the top $25\%$ or even the top $1\%$) also have a relatively high number of incoming edges when compared to the rest of the network. It thus seems that pages that have a lot of links to other pages also seem to be relatively high linked (i.e. have incoming edges) pages themselves  Note that there is a relatively large group (i.e. pages). The same can be seen in the top groups of when grouped/sorted by number of incoming edges. These groups do not only have a relatively higher number of incoming edges (on average) than the rest of the network, but also a relatively higher frequency of   in the network that do not have incoming edges, but have a relatively high number of outgoing edges.

In [27]:
print(str((wiki_df.loc[wiki_df["out edges"] == 0.00].shape[0] / wiki_df["out edges"].shape[0]) * 100), "%")
print(str((wiki_df.loc[wiki_df["in edges"] == 0.00].shape[0] / wiki_df["in edges"].shape[0]) * 100), "%")

29.691253735784134 %
29.83103724074137 %


In addition, one can see that part of the skewness of the degree distribution (see bottom groups) is due to pages that have either no incoming or outgoing edges. Having no incoming edges can be explained by pages that are the "entry-page" to this network of pages (i.e. referred to by somebody googling it / typing it in their address bar) while having no outgoing edges can be explained by pages that simply have no links on them (i.e. pages showing files etc.). These possible explanations are, however, just assumptions / guesses. There is no metadata about the pages available, so we can actually not confirm these hypotheses by more in-depth evaluations.

### <font color="darkgreen">2. Static Network Analysis:</font> Diameter

The next meausure which is going to be calculated and evaluated in the context of static network analysis is a grpah distance measure: the diameter. It encapsulates the maximum found eccentricity of any node in the network. Or in other words the longest of all the shortest paths between the vertices in the network. This may sound a bit confusing, but imagine that we have a set of all the shortest paths between each pair of vertices in the network. The maximum found value in this set is the diameter of the network. 

<b>A</b> --- <b>B</b> --- <b>C</b>&emsp;&emsp;&emsp; <i>Diameter:</i> $4$ (A-I and G-C) <br> 
|&emsp;&emsp;|&emsp;&emsp;| <br>
<b>D</b> --- <b>E</b> --- <b>F</b> <br>
|&emsp;&emsp;|&emsp;&emsp;| <br>
<b>G</b> --- <b>H</b> --- <b>I</b> 

<b>A</b> --- <b>B</b> --- <b>C</b>&emsp;&emsp;&emsp; <i>Diameter:</i> $5$  (A-J) <br> 
|&emsp;&emsp;|&emsp;&emsp;| <br>
<b>D</b> --- <b>E</b> --- <b>F</b> <br>
|&emsp;&emsp;|&emsp;&emsp;| <br>
<b>G</b> --- <b>H</b> --- <b>I</b> --- <b>J</b> 

This toyish example should give you a clear overview of what is measured by the distance measure that is diameter. Note, however, that this example is considering undirected edges, while we are of course interested in this measure of a directed graph of links. We will translate the measure back to our problem and network context in a bit.

Since we have a directed graph (i.e. network) that is not strongly connected since there are nodes $u$ which have no path to certain nodes $v$, we cannot simply use the diameter function of $networkx$ to calculate the diameter of the graph. The reason for this is a that if a shortest path from $u$ to $v$ is non-existent in the graph, it will be $\infty$ (think about Dijkstra's shortest path algorithm). And since the diameter function of $networkx$ simply calcultes the eccentricity of the Graph and returns the max value found in this collection, it will not work since the eccentricity function and thus the diameter function will raise an error because the graph is not strongly connected. 

In [28]:
nx.is_strongly_connected(G)

False

The problem with the built in $networkx$ function for calculating the diameter of a graphh is that the function relies on the eccentricity function. Eccentricity finds the shortest path from a node $u$ to all the other nodes and does this this for all the graphs in the network. The diameter function then calls this function on the graph and takes the max (i.e. longest shortest path) over the returned collection of all the shortest paths by eccentricity. 

In [38]:
strong_comp = G_i.clusters(mode = 'STRONG')
strong_comp_items = list(strong_comp)
len(strong_comp_items)

847138

There are a lot of self loops and mutual connections as we wil see in the next section. This results in strong components of one or two elements, which are not very useful in this stage. We can excude the components of length $\leq 2$

In [39]:
strong_comp_items_substantial = [component for component in strong_comp_items if len(component) > 2]
len(strong_comp_items_substantial)

209

This results in 209 strong components in the graph. Let's see what the largest strong component is in the graph and try to obtain the diameter of this strong component, for the largest diameter value among the strong components. We can start by excluding more strong components since there are only a few large components in the current collection (i.e. still a lot of relatively small components that will probably not yield the largest diameter of all the strong components of the graph)

In [40]:
strong_comp_items_substantial = [component for component in strong_comp_items_substantial if len(component) > 100]
len(strong_comp_items_substantial)

1

That is better. Let's asses what the size of this large strong component is (i.e. a subgraph where each vertex is connected to each other vertex in the graph).

In [36]:
component = strong_comp_items_substantial[0]
len(component)

188879

So even if we would try to reduce the original graph to a strong connected graph (e.g. the largest stongh graph in the network, it would still be infeasible to calculate it for a strongly connected subgraph by simply exploiting (non-heurisitc) algorithms found in network/graph libraries written in Python/C++ without exploiting more advanced computing mehtodologies (e.g. parallel computing etc.)

The implementations of the libraries of networkx and igraph need a strong graph as input. The graph was, obviously, not strongly connected. This would not make sense of course for a network representing a network of pages of over 600.000 nodes. Having a strongly connected graph would implicate that there was a link from every page to every other page in the network. Something that you will never encounter in such a network of this magnitude which represents such a domain network.

When considering the common definition of the diameter of a (directed) graph one often finds something in the sense of:

<i>"As another means of measuring network graphs, we can define the diameter of a network as the longest of all the calculated shortest paths in a network. It is the shortest distance between the two most distant nodes in the network. In other words, once the shortest path length from every node to all other nodes is calculated, the diameter is the longest of all the calculated path lengths. The diameter is representative of the linear size of a network."</i>

So the diameter that has been calculated right now is not really in line with what should be encapsulated by the diameter value. We are interested in the longest shortest path in the graph. Or in other words the shortest path between the two most distant node in the graph.

An exhaustive search for this value is not considered possbile with our resources and time-schedule. Such an algorithm should iterate over $6.7 * 10^5$ nodes, find all of their successors by either exploiting bread-first search or depth-first search based algorithms/variations and calculate the shortest path between each node and its succesors. Just to give you an impression of the computational workload of such a task: 

In [41]:
len(dict(nx.bfs_successors(G, '7')))

77526

By having nodes with such large number of successors, it is considered infeasible to calculate this by simple algorithms of Python libraries without exploiting more advanced computing methods. Several variations of algorithms of the networkx, igraph libraries and even the graph-tool library (more efficient since the core data-structures and algorithms are implemented in C++) have been tested for computational feasbility of this task. We even tried to change the source-code to implement implementations that we have devised ourselves for this network. These optimizations were based on the following observations:

In [42]:
wiki_df.loc[((wiki_df['out edges'] == 0) | (wiki_df['out edges'] == 1))].shape[0]

372221

The thing worht nothing is the large quantity of nodes with only one outgoing edge or no outgoing edge. This fact can be exploited by skipping nodes with an out_degree of 0 and remembering nodes of degree 1 that were visited in previous iterations over the successors of previous nodes that were considered. These nodes can be skipped in the subsequent checks since considering their successors will never yield the true diameter of the graph. The reason for this is that if this node is a successor of another node, the longest shortest path found for this node among its successors will always be $\geq$ than the longest shortest path that will be found by exploring the shortest path to the successors of the node of degree $1$ since if we assume that this path would be a longer shortest path, a longest path found by going through that node of out-degree 1 will be at least as long. This observation was exploited in one of the optimizations below. Other optimizations that were exploited invovled implementing/changing the  bread-first search of networkx to prune paths that do not lead to a longer shortest path. We, however, want to limit the amount of code somewhat in this analysis, so the code of only the first optimization is devised below.

First note that since the function $nx.shortest\_path\_length$ relies on an implementation of Dijkstra's algorithm, the choice is made to remove the selfloop-edges since this (can) corrupt the results and the running time of the algorithm depending on the pecularities of the implementation of the algorithm.

In [43]:
self_loop_edges = list(nx.selfloop_edges(G))
edges_no_loop = list(set(G.edges()).difference(set(self_loop_edges)))
G_no_loop = G.edge_subgraph(edges_no_loop)
len(self_loop_edges) + len(edges_no_loop)== len(G.edges())

True

In [44]:
already_visited_node = set() # global tracker of nodes of out-degree one that are already visited

Calculating the longest shortest path in the network beginning from source node $u$ by iterating over the successors. And facilitating the optimization in another function by storing nodes of out-degree 1 that were already visited.

In [45]:
def longest_shortest_path(G_in, u):
    global already_visited_node
    longest = tracker = 0
    reachable = set(nx.dfs_successors(G_in, source = u))
    reachable = list(reachable - set(u))
    for v in reachable: # depth-first-search of successors
        #print("successor: ", tracker); tracker += 1
        if ((u != v)):
            if (G.out_degree(v) == 1):
                already_visited_node.add(v)
            path_length = nx.shortest_path_length(G_in, source = u, target = v)
            if (path_length > longest):
                longest = path_length
    return longest

In [46]:
def calculate_diameter(G_in):
    global already_visited_node
    longest = tracker = 0
    nodes = list(G_in)
    for node in nodes:
        # print("node: ", tracker); tracker += 1
        if ((G_in.out_degree(node) != 0) and (node not in already_visited_node)):
            longest_shortest = longest_shortest_path(G_in, node)
            if (longest_shortest > longest):
                longest = longest_shortest
    return longest

The function could be optimized even more by implementing breadth-first-search ourselves and comparing the shortest path-lengths along the way, but thay may be a bit far fetched for this analysis. 

Assuming having a (network) of computers capable of calculating the diameter of this graph in a reasonable time one can use a function call $diameter\,=\,longest\_shortest\_path(G)$ to do so. 

This calculation will take, however, a very long time and we will, therefore, refrain ourselves from these (type of) measures in the rest of the analysis since we have covered more than enough other measures (even though we only had to cover eight measures). Benchmarking this function for a smaller (sub)graph agains the standard diameter functions of networkx, igraph and graph-tool or a similar self-written implementation without the optimization shows indeed that there is a significant improvement in the computation time by these optimizations.

The following function can be used to actually fill the $wiki\_df$ DataFrame with relevant distance measures such as the longest shortest path found amoung the successors of a certain node (i.e. row in the table), the number of successors and the mean shortest path among the sucessors of a certain node. The longest shortest path or the mean shortest path of the graph can then be obtained by simply querying the DataFrame.

In [47]:
tracker = 0

In [48]:
def nr_successors_and_diameter_and_mean(node):
    node = str(node)
    global tracker # track progress
    total = mean = longest = nr_paths = 0 
    successors = set(nx.dfs_successors(G, node))
    successors = list(successors - (set(node)))
    
    for v in successors:
        print("node: ", tracker)
        tracker += 1
        path_length = nx.shortest_path_length(G, source = node, target = v)
        nr_paths += 1
        total += path_length
        if (path_length > longest):
            longest = path_length
    if (nr_paths != 0):
        mean = int(total / nr_paths)
    return longest, mean, len(successors)

In [49]:
# wiki_df['longest_sp'], wiki_df['mean_sp'], wiki_df['nr_successors'] = zip(*wiki_df['node'].map(nr_successors_and_diameter_and_mean)) 

Even though we are not able to do the compuations over all the nodes, we can get a rough estimate of the magnitude (i.e. at least a minimum) of the diameter of the graph by doing the longest shortest-path computations of a set of nodes. Let's see what the results are for nodes with a relatively high out-degree.

In [59]:
top10_outgoing_links = outgoing_edges_top_1.iloc[-10:]
top10_outgoing_links

,node,in edges,out edges,decile_incoming_edges,decile_outgoing_edges
678897,970034,0,2280,0,5
678898,16115,4,2336,0,5
678899,598952,0,2351,0,5
678900,707633,0,2386,0,5
678901,9273,0,2579,0,5
678902,707642,0,2620,0,6
678903,377365,0,3173,0,7
678904,9258,37,3694,0,8
678905,154856,8,3951,0,9
678906,306188,0,4302,0,9


In [ ]:
for node in list(top10_outgoing_links.node):
    longest_shortest_path_found = longest_shortest_path(G, node)
    print("node: ", node, "with longest shortest path of ", longest_shortest_path_found, " among its large number of successors")

node:  970034 with longest shortest path of  18  among its large number of successors
node:  16115 with longest shortest path of  18  among its large number of successors
node:  598952 with longest shortest path of  17  among its large number of successors
node:  707633 with longest shortest path of  18  among its large number of successors
node:  9273 with longest shortest path of  18  among its large number of successors
node:  707642 with longest shortest path of  18  among its large number of successors


In [54]:
print(longest_shortest_path(G, '394')) # longest shortest path of a few random nodes as source
print(longest_shortest_path(G, '300'))
print(longest_shortest_path(G, '7'))
print(longest_shortest_path(G, '42'))
print(longest_shortest_path(G, '9643'))

17

This (evaluation of the nodes with the largest out-degree) should give us a good insight on the longest shortest path found in the graph, even though it can be of course that the actual longest shortest path of the graph is found by exploring the successors of other nodes with a lower number of outgoing edges on the first 'level'. There is, however, a high probability that these maximum of these values is the actual longest shortest path found in the graph and thus the actual diameter of the graph.

Note that this network propert is an intensively researched field in the world of network science. Searching content in the world-wide-web is related to the diameter of this network. One can grasp this content by imagining it as the number of clicks one would have to make through all the links or the amount of links a search-bot has to explore. The diameter of the network cannot simply be computed by an exhaustive brute-force search of such large networks ss explained in the paper <a href="http://people.math.sc.edu/lu/papers/pdiam.pdf">The Diameter of Random Massive Graphs, Linyuan Lu, Oct 2000"</a>, many massive graphs (i.e. networks) share certain universal characteristics which can be used to describe these networks in term of so-called power-laws. The actual implications of the found values in this network shed light on how many 'clicks' a potential visitor can though at maximum when visitin a certain topic. It can, for example, shed light on the number of clicks that can be done by exploring the links that go from one (sub-)topic to another (sub-)topic, by following the minimum path possible in the network, and without revisiting the same node twice. This is just a thought and has to be supported by further analysis with page meta-data if one actually would like to evaluate such a hypothesis.

### <font color="darkgreen">3. Static Network Analysis:</font> Dyads

This might be an uncommon measure in this course, but it can have quite interesting implications for the network in context. We wanted to go beyond the suggested measures and have, therefore, done some reading on networks science ourselves. One of the papers we came accross was an elaboration on Dyads and Triads found in Directed Graphs. It boils down to classifying each pair of nodes $u$ and $v$ in the network (i.e. a directed graph in our context) into three categories, which are:

1. <b>Mutual:</b> There is an edge from $u$ to $v$ and also an edge frome $v$ to $u$
2. <b>Asymmetric:</b> There is either an edge from $u$ to $v$ or an edge from $v$ to $u$
3. <b>Null:</b> There is no edge from $u$ to $v$ and also no edge frome $v$ to $u$

The actual implications for our network should be quite evident. We can measure how many pages have a symmetric link between each other, how many pages have a one way link to each other and how many pages have no link between each other. We are going to  elaborate on this further on after some calculations.

In [ ]:
graph_dyad_census = G_i.dyad_census()

In [ ]:
print(sorted(graph_dyad_census.as_dict().items()))

Note, however, that this includes nodes with self-loops and parallel edges as explained before that deterioriate any observations that can be made about the measures, so before we actually place it in the problem context, let's remove those edges and recalcuate the measures.

In [ ]:
G_i = G_i.simplify(multiple = True, loops = True, combine_edges = None)

In [ ]:
graph_dyad_census = G_i.dyad_census()

In [ ]:
print(sorted(graph_dyad_census.as_dict().items()))

To put this in better perspective it makes sense to look at the percentages of the total $u \times v$ pairs which are possible in the network. And even more common is to look at this number compared to the total number of edges, as we will also see in the evalauation of another metric in the next section.

In [ ]:
total_count = sum(list(graph_dyad_census))
number_edges = len(G_i.es)

for metric in sorted(graph_dyad_census.as_dict().items())[:2]:
    if(metric[0] == 'mutual'): # only one of the edges is counted for mutual connnections
        print("percentage of total possible combinations for ", metric[0], " edges, is: ", round(((metric[1] * 2 / total_count) * 100), 4), "%")
        print("percentage of total number of edges for ", metric[0], " edges, is: ", round(((metric[1] * 2/ number_edges) * 100), 4), "%\n")
    else :
        print("percentage of total possible combinations for ", metric[0], " edges, is: ", round(((metric[1] / total_count) * 100), 4), "%")
        print("percentage of total number of edges for ", metric[0], " edges, is: ", round(((metric[1] / number_edges) * 100), 4), "%\n")

There are thus 1) a lot of pages which do not have an (asymmetric) link to each other, and the number of pages that link to each other (i.e. mutual) are even less common. This actually does make sense when one considers the context of a network of a website/domain. One would not expect that every page has a link to every other page in the network, especially considering the size of this particular network. In addition, having mutual edges on a website depends entirely on the type of webpages one is considering. For some webpages it makes sense considering the navigation of the website. Or consider pages on WikiPedia which refer to a sub-topic of certain topic. This sub-topic is likely to have also a link back to the main topic.

Consider, for example, the WikiPedia page on Network Science. Several links to sub-topics and/or related topics have a link on the page back to the page of Network Science. This is not only due to reference links in the text (i.e. information) on that page, but also due to the navigation lay-out of wikipedia which enhances this type of navigation by suggesting similar / related topics and classifiying (sub-)topics into groups (see the Wiki page on Complex Networks for example).

One could thus consider the measure to be an indicator of how related certain pages are if one considers this context. This can be interesting to evaulate over time (i.e task 2). We can test this hypothesis of the relation between mutual edges and relations between (sub)-topics by analyzing both the relative and absolute changes of these measures. Note, however, that we cannot confirm these hypotheses by the lack of context (i.e. metadata about the pages). But first, we can go into a little more depth about the implications of these measures and also consider another, related, measure in the next section.

At the moment, nodes with no incoming edges are included. The same goes for pages with no outgoing links. The question is, however, if we want to include these pages in our results. If we place the network in question again in the context of its origin one would assume that these pages can be considered as 'entry'-pages to the domain of the network while the nodes with no outgoing links can be pages with only links to pages outside of the domain. Wikipedia has, for example, a lot of pages with media (i.e. pictures) that only have a link to the source of the content (i.e. links to twitter, wikimediafoundation, google images etc.). These pages are not really interesting to for the assessment of the mutuality hypotheses. Let's re-asses the values once more after the removal of these nodes.

In [ ]:
to_delete_ids = [v.index for v in G_i.vs if ((v.indegree() == 0) | (v.outdegree == 0))]

In [ ]:
G_i.delete_vertices(to_delete_ids)

In [ ]:
graph_dyad_census = G_i.dyad_census()
print(sorted(graph_dyad_census.as_dict().items())[:3])
total_count = sum(list(graph_dyad_census))
number_edges = len(G_i.es)

for metric in sorted(graph_dyad_census.as_dict().items())[:2]:
    if(metric[0] == 'mutual'): # only one of the edges is counted for mutual connnections
        print("percentage of total possible combinations for ", metric[0], " edges, is: ", round(((metric[1] * 2 / total_count) * 100), 4), "%")
        print("percentage of total number of edges for ", metric[0], " edges, is: ", round(((metric[1] * 2/ number_edges) * 100), 4), "%\n")
    else :
        print("percentage of total possible combinations for ", metric[0], " edges, is: ", round(((metric[1] / total_count) * 100), 4), "%")
        print("percentage of total number of edges for ", metric[0], " edges, is: ", round(((metric[1] / number_edges) * 100), 4), "%\n")

Which gives a better representation of the measures in our particular network. In conclusion, we can consider the 'dyad classification' as a representation of the cohesion of the network (i.e. how strong is the tendency to have a returning link on when visiting a succesive node in the network).

### <font color="darkgreen">4. Static Network Analysis:</font> Reciprocity and Transitivity

In the previous section, dyad measures were introduced and we had a look at the percentage of the mutual  and asymmetric edges in the graph compared to the total number of possible combinations in the graph (i.e. $(n-1)^2$ where $n$ is the number of vertices (no self-loops)) and compared to the number of edges in the graph. This last measure is often refered to as the dyad-based reciprocity of the network.